In [ ]:
pip install pygame

In [ ]:
import pygame
import sys

class RunGame:
    def __init__(self):
        pygame.init()
        self.screen = pygame.display.set_mode((800, 600))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 36)
        self.running = False
        self.car = None

    def menu(self):
        track_edit_box = pygame.Rect(300, 200, 200, 50)
        car_edit_box = pygame.Rect(300, 300, 200, 50)
        load_game_box = pygame.Rect(300, 400, 200, 50)

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    mouse_pos = event.pos
                    if track_edit_box.collidepoint(mouse_pos):
                        print("Opening Track Editor")
                        # Implement: trackEdit = trackEditor()
                    elif car_edit_box.collidepoint(mouse_pos):
                        print("Opening Car Editor")
                        # Implement: carEdit = car()
                    elif load_game_box.collidepoint(mouse_pos):
                        print("Loading Game")
                        find_file = input("Enter file name to load: ")
                        self.load_file(find_file)

            self.screen.fill((255, 255, 255))  # White background

            pygame.draw.rect(self.screen, (200, 200, 200), track_edit_box)
            pygame.draw.rect(self.screen, (200, 200, 200), car_edit_box)
            pygame.draw.rect(self.screen, (200, 200, 200), load_game_box)

            track_text = self.font.render('Make Track', True, (0, 0, 0))
            car_text = self.font.render('Make Car', True, (0, 0, 0))
            load_text = self.font.render('Load Track', True, (0, 0, 0))

            self.screen.blit(track_text, (track_edit_box.x + 20, track_edit_box.y + 10))
            self.screen.blit(car_text, (car_edit_box.x + 30, car_edit_box.y + 10))
            self.screen.blit(load_text, (load_game_box.x + 20, load_game_box.y + 10))

            pygame.display.flip()
            self.clock.tick(60)

    def load_file(self, file_name):
        # Implement file loading logic here
        print(f"Loading file: {file_name}")
        self.running = True
        # Implement: self.car = loadCarDetails

if __name__ == "__main__":
    game = RunGame()
    game.menu()

In [ ]:
import pygame
import sys


class Grid:
    def __init__(self, size, screen_size):
        self.size = size
        self.cell_size = screen_size // size
        self.grid = [[None for _ in range(size)] for _ in range(size)]

    def get_cell(self, x, y):
        grid_x = x // self.cell_size
        grid_y = y // self.cell_size
        return grid_x, grid_y

    def place_block(self, x, y, block):
        self.grid[y][x] = block

    def remove_block(self, x, y):
        self.grid[y][x] = None

    def draw(self, screen):
        for y in range(self.size):
            for x in range(self.size):
                rect = pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
                pygame.draw.rect(screen, (200, 200, 200), rect, 1)
                if self.grid[y][x]:
                    pygame.draw.rect(screen, self.grid[y][x], rect)

class TrackEditor:
    def __init__(self):
        pygame.init()
        self.screen_size = 800
        self.screen = pygame.display.set_mode((self.screen_size, self.screen_size))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 24)

        self.grid_size = self.get_grid_size()
        self.grid = Grid(self.grid_size, self.screen_size)

        self.blocks = [
            (255, 0, 0),    # Red
            (0, 255, 0),    # Green
            (0, 0, 255),    # Blue
            (255, 255, 0),  # Yellow
            (255, 0, 255),  # Magenta
        ]
        self.selected_block = self.blocks[0]  # Start with the first block selected
        self.palette_rect = pygame.Rect(self.screen_size - 80, 0, 80, self.screen_size)

    def get_grid_size(self):
        size = int(input("Enter grid size (e.g., 20 for a 20x20 grid): "))
        return size

    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    self.handle_click(event.pos, event.button)
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_s:
                        self.save_track()
                    elif event.key == pygame.K_l:
                        self.load_track()

            self.screen.fill((0, 255, 0))  # Green background
            self.grid.draw(self.screen)
            self.draw_block_palette()
            self.draw_instructions()
            pygame.display.flip()
            self.clock.tick(60)

        pygame.quit()

    def handle_click(self, pos, button):
        if not self.palette_rect.collidepoint(pos):
            grid_x, grid_y = self.grid.get_cell(pos[0], pos[1])
            if 0 <= grid_x < self.grid_size and 0 <= grid_y < self.grid_size:
                if button == 1:  # Left click
                    if self.selected_block:
                        self.grid.place_block(grid_x, grid_y, self.selected_block)
                    else:
                        self.grid.remove_block(grid_x, grid_y)
                elif button == 3:  # Right click
                    self.grid.remove_block(grid_x, grid_y)
        else:
            self.handle_palette_click(pos)

    def handle_palette_click(self, pos):
        index = (pos[1] - 20) // 40
        if 0 <= index < len(self.blocks):
            self.selected_block = self.blocks[index]
        elif index == len(self.blocks):
            self.selected_block = None  # Eraser

    def draw_block_palette(self):
        pygame.draw.rect(self.screen, (255, 255, 255), self.palette_rect)
        for i, color in enumerate(self.blocks):
            rect = pygame.Rect(self.screen_size - 60, 20 + i * 40, 40, 30)
            pygame.draw.rect(self.screen, color, rect)
            if color == self.selected_block:
                pygame.draw.rect(self.screen, (255, 255, 255), rect, 2)
        rect = pygame.Rect(self.screen_size - 60, 20 + len(self.blocks) * 40, 40, 30)
        pygame.draw.rect(self.screen, (0, 0, 0), rect)  # Eraser
        if self.selected_block is None:
            pygame.draw.rect(self.screen, (255, 255, 255), rect, 2)

    def draw_instructions(self):
        text = self.font.render("Left-click to place/remove blocks", True, (0, 0, 0))
        self.screen.blit(text, (10, 10))
        text = self.font.render("Right-click to remove blocks", True, (0, 0, 0))
        self.screen.blit(text, (10, 30))
        text = self.font.render("Press 'S' to save track", True, (0, 0, 0))
        self.screen.blit(text, (10, 50))
        text = self.font.render("Press 'L' to load track", True, (0, 0, 0))
        self.screen.blit(text, (10, 70))

    def save_track(self):
        file_name = input("Enter file name: ")
        with open(file_name + '.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            for y in range(self.grid_size):
                for x in range(self.grid_size):
                    block = self.grid.get_block(x, y)
                    if block:
                        writer.writerow([x, y, block])

    def load_track(self):
        file_name = input("Enter file name: ")
        if os.path.exists(file_name + '.csv'):
            self.grid.clear()
            with open(file_name + '.csv', 'r') as file:
                reader = csv.reader(file)
                for row in reader:
                    x, y, block = row
                    self.grid.place_block(int(x), int(y), tuple(map(int, block[1:-1].split(','))))
        else:
            print("File not found!")

if __name__ == "__main__":
    track_editor = TrackEditor()
    track_editor.run()

In [9]:
import pygame
import sys
import csv
import os
import math

pygame 2.6.1 (SDL 2.28.4, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
class Grid:
    def __init__(self, size, screen_size):
        self.size = size
        self.cell_size = screen_size // size
        self.grid = [[None for _ in range(size)] for _ in range(size)]

    def get_cell(self, x, y):
        grid_x = x // self.cell_size
        grid_y = y // self.cell_size
        return grid_x, grid_y

    def place_block(self, x, y, block):
        self.grid[y][x] = block

    def remove_block(self, x, y):
        self.grid[y][x] = None

    def get_block(self, x, y):
        return self.grid[y][x]

    def clear(self):
        self.grid = [[None for _ in range(self.size)] for _ in range(self.size)]

    def draw(self, screen):
        for y in range(self.size):
            for x in range(self.size):
                rect = pygame.Rect(x * self.cell_size, y * self.cell_size, self.cell_size, self.cell_size)
                pygame.draw.rect(screen, (200, 200, 200), rect, 1)
                if self.grid[y][x]:
                    pygame.draw.rect(screen, self.grid[y][x], rect)

In [2]:
class TrackEditor:
    def __init__(self, screen):
        self.screen = screen
        self.screen_width = screen.get_width()
        self.screen_height = screen.get_height()
        self.grid_pixel_size = min(self.screen_width, self.screen_height) - 100  # Leave some space for UI
        self.clock = pygame.time.Clock()
        self.font = pygame.font.Font(None, 24)
    
        self.grid_size = self.get_grid_size()
        self.init_grid()
    
        self.blocks = [
            (255, 0, 0),    # Red
            (0, 255, 0),    # Green
            (0, 0, 255),    # Blue
            (255, 255, 0),  # Yellow
            (255, 0, 255),  # Magenta
        ]
        self.selected_block = self.blocks[0]  # Start with the first block selected
        self.palette_rect = pygame.Rect(self.screen_width - 80, 0, 80, self.screen_height)

    def init_grid(self):
        self.grid = Grid(self.grid_size, self.grid_pixel_size)
        
    def get_grid_size(self):
        size = int(input("Enter grid size (e.g., 20 for a 20x20 grid): "))
        return size

    def run(self):
        running = True
        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    return "QUIT"
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    self.handle_click(event.pos, event.button)
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_s:
                        self.save_track()
                    elif event.key == pygame.K_l:
                        self.load_track()
                    elif event.key == pygame.K_ESCAPE:
                        return "MENU"

            self.screen.fill((200, 200, 200))  # Light gray background
            
            # Calculate the position to center the grid
            grid_x = (self.screen_width - self.grid_pixel_size) // 2
            grid_y = (self.screen_height - self.grid_pixel_size) // 2
            
            # Create a surface for the grid
            grid_surface = pygame.Surface((self.grid_pixel_size, self.grid_pixel_size))
            grid_surface.fill((0, 255, 0))  # Green background for the grid
            self.grid.draw(grid_surface)
            
            # Draw the grid surface on the main screen
            self.screen.blit(grid_surface, (grid_x, grid_y))
            
            self.draw_block_palette()
            self.draw_instructions()
            pygame.display.flip()
            self.clock.tick(60)

    def handle_click(self, pos, button):
        if self.palette_rect.collidepoint(pos):
            # Handle palette click
            self.handle_palette_click(pos)
        else:
            grid_x = (self.screen_width - self.grid_pixel_size) // 2
            grid_y = (self.screen_height - self.grid_pixel_size) // 2
            
            if grid_x <= pos[0] < grid_x + self.grid_pixel_size and grid_y <= pos[1] < grid_y + self.grid_pixel_size:
                cell_x, cell_y = self.grid.get_cell(pos[0] - grid_x, pos[1] - grid_y)
                if 0 <= cell_x < self.grid_size and 0 <= cell_y < self.grid_size:
                    if button == 1:  # Left click
                        if self.selected_block:
                            self.grid.place_block(cell_x, cell_y, self.selected_block)
                        else: # Remove block
                            self.grid.remove_block(cell_x, cell_y)
                    elif button == 3:  # Right click
                        self.grid.remove_block(cell_x, cell_y)

    def handle_palette_click(self, pos):
        index = (pos[1] - 20) // 40
        if 0 <= index < len(self.blocks):
            self.selected_block = self.blocks[index]
        elif index == len(self.blocks):
            self.selected_block = None  # Eraser

    def draw_block_palette(self):
        pygame.draw.rect(self.screen, (255, 255, 255), self.palette_rect)
        for i, color in enumerate(self.blocks):
            rect = pygame.Rect(self.screen_width - 60, 20 + i * 40, 40, 30)
            pygame.draw.rect(self.screen, color, rect)
            if color == self.selected_block:
                pygame.draw.rect(self.screen, (255, 255, 255), rect, 2)
        rect = pygame.Rect(self.screen_width - 60, 20 + len(self.blocks) * 40, 40, 30)
        pygame.draw.rect(self.screen, (0, 0, 0), rect)  # Eraser
        if self.selected_block is None:
            pygame.draw.rect(self.screen, (255, 255, 255), rect, 2)

    def draw_instructions(self):
        text = self.font.render("Left-click to place/remove blocks", True, (0, 0, 0))
        self.screen.blit(text, (10, 10))
        text = self.font.render("Right-click to remove blocks", True, (0, 0, 0))
        self.screen.blit(text, (10, 30))
        text = self.font.render("Press 'S' to save track", True, (0, 0, 0))
        self.screen.blit(text, (10, 50))
        text = self.font.render("Press 'L' to load track", True, (0, 0, 0))
        self.screen.blit(text, (10, 70))
        text = self.font.render("Press 'Esc' to return to menu", True, (0, 0, 0))
        self.screen.blit(text, (10, 90))

    def save_track(self):
        file_name = input("Enter file name: ")
        with open(file_name + '.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            # Write the grid size as the first row
            writer.writerow([self.grid_size])
            for y in range(self.grid_size):
                for x in range(self.grid_size):
                    block = self.grid.get_block(x, y)
                    if block:
                        writer.writerow([x, y, block])

    def load_track(self):
        file_name = input("Enter file name: ")
        if os.path.exists(file_name + '.csv'):
            with open(file_name + '.csv', 'r') as file:
                reader = csv.reader(file)
                # Read the grid size from the first row
                self.grid_size = int(next(reader)[0])
                # Reinitialize the grid with the new size
                self.init_grid()
                for row in reader:
                    x, y, block = row
                    self.grid.place_block(int(x), int(y), tuple(map(int, block[1:-1].split(','))))
            return self.grid  # Return the loaded grid
        else:
            print("File not found!")
            return None

In [26]:
class Car:
    def __init__(self, x, y, width, height):
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.angle = 0
        self.speed = 0
        self.acceleration = 0.2
        self.deceleration = 0.1
        self.max_speed = 5
        self.turn_speed = 3
        self.color = (255, 0, 0)  # Red car

    def accelerate(self):
        self.speed += self.acceleration
        if self.speed > self.max_speed:
            self.speed = self.max_speed

    def decelerate(self):
        self.speed -= self.acceleration
        if self.speed < -self.max_speed / 2:  # Slower in reverse
            self.speed = -self.max_speed / 2

    def turn_left(self):
        self.angle += self.turn_speed

    def turn_right(self):
        self.angle -= self.turn_speed

    def update(self):
        # Move the car based on its speed and angle
        self.x += self.speed * math.cos(math.radians(self.angle))
        self.y -= self.speed * math.sin(math.radians(self.angle))
        
        # Apply friction
        if self.speed > 0:
            self.speed -= self.deceleration
        elif self.speed < 0:
            self.speed += self.deceleration
        
        # Stop the car if speed is very low
        if abs(self.speed) < self.deceleration:
            self.speed = 0

    def draw(self, screen):
        # Create a surface for the car
        car_surface = pygame.Surface((self.width, self.height), pygame.SRCALPHA)
        pygame.draw.rect(car_surface, self.color, (0, 0, self.width, self.height))
        
        # Rotate the car surface
        rotated_car = pygame.transform.rotate(car_surface, self.angle)
        
        # Get the rect of the rotated car and set its center
        rect = rotated_car.get_rect(center=(self.x, self.y))
        
        # Draw the rotated car on the screen
        screen.blit(rotated_car, rect.topleft)

In [23]:
class Track:
    def __init__(self, name):
        self.name = name
        self.blocks = []
        self.grid_size = 0
        self.block_size = 0
        self.screen_size = 800  # We'll use a square screen
        self.load_track()

    def load_track(self):
        try:
            with open(f"{self.name}.csv", 'r') as file:
                reader = csv.reader(file)
                # First row contains the grid size
                self.grid_size = int(next(reader)[0])
                self.block_size = self.screen_size // self.grid_size

                for row in reader:
                    if len(row) == 3:
                        x, y, color = int(row[0]), int(row[1]), eval(row[2])
                        self.blocks.append((x, y, color))
        except FileNotFoundError:
            print(f"Track file '{self.name}.csv' not found.")
        except Exception as e:
            print(f"Error loading track: {str(e)}")

    def draw(self, screen):
        # Fill the background with green
        screen.fill((0, 200, 0))  # Green color
        
        for block in self.blocks:
            x, y, color = block
            pygame.draw.rect(screen, color, (x * self.block_size, y * self.block_size, self.block_size, self.block_size))

    def get_car_size(self):
        # Make the car size proportional to the grid
        car_width = self.block_size * 0.5  # 50% of a block width
        car_height = self.block_size * 0.25  # 25% of a block height
        return car_width, car_height

    def get_screen_size(self):
        return self.screen_size, self.screen_size

In [5]:
class Menu:
    def __init__(self, screen):
        self.screen = screen
        self.font = pygame.font.Font(None, 36)
        self.options = ["Track Editor", "Car Editor", "Load Game", "Quit"]
        self.selected = 0

    def draw(self):
        self.screen.fill((255, 255, 255))
        for i, option in enumerate(self.options):
            color = (255, 0, 0) if i == self.selected else (0, 0, 0)
            text = self.font.render(option, True, color)
            self.screen.blit(text, (100, 100 + i * 50))
        pygame.display.flip()

    def handle_input(self, event):
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                self.selected = (self.selected - 1) % len(self.options)
            elif event.key == pygame.K_DOWN:
                self.selected = (self.selected + 1) % len(self.options)
            elif event.key == pygame.K_RETURN:
                return self.options[self.selected]
        return None

In [24]:
def drive_car(screen, track_name):
    track = Track(track_name)
    screen_size = track.get_screen_size()
    screen = pygame.display.set_mode(screen_size)  # Resize the screen
    
    car_width, car_height = track.get_car_size()
    
    # Place the car at the center of the screen
    car_x = (screen_size[0] - car_width) / 2
    car_y = (screen_size[1] - car_height) / 2
    
    car = Car(car_x, car_y, car_width, car_height)

    clock = pygame.time.Clock()
    running = True
    
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return "QUIT"
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    return "MENU"

        keys = pygame.key.get_pressed()
        if keys[pygame.K_w]:
            car.accelerate()
        if keys[pygame.K_s]:
            car.decelerate()
        if keys[pygame.K_a]:
            car.turn_left()
        if keys[pygame.K_d]:
            car.turn_right()

        car.update()

        track.draw(screen)
        car.draw(screen)
        
        pygame.display.flip()
        clock.tick(60)

    return "MENU"

In [19]:
def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 800))  # Start with a square screen
    pygame.display.set_caption("Racing Game")
    clock = pygame.time.Clock()

    menu = Menu(screen)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            else:
                option = menu.handle_input(event)
                if option:
                    if option == "Track Editor":
                        track_editor = TrackEditor(screen)
                        result = track_editor.run()
                        if result == "QUIT":
                            pygame.quit()
                            sys.exit()
                        elif result == "MENU":
                            menu = Menu(screen)
                    elif option == "Car Editor":
                        # Implement car editor
                        pass
                    elif option == "Load Game":
                        track_name = input("Enter track name to load: ")
                        result = drive_car(screen, track_name)
                        if result == "QUIT":
                            pygame.quit()
                            sys.exit()
                        elif result == "MENU":
                            # Reinitialize the menu with the original screen size
                            screen = pygame.display.set_mode((800, 800))
                            menu = Menu(screen)
                    elif option == "Quit":
                        pygame.quit()
                        sys.exit()

        menu.draw()
        pygame.display.flip()
        clock.tick(60)

In [25]:
if __name__ == "__main__":
    main()

Enter track name to load:  test


SystemExit: 